In [10]:
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import s3fs
import scipy
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf

sys.path.append("/Users/fgu/dev/projects/entropy")
import entropy.data.aggregators as ag
import entropy.data.cleaners as cl
import entropy.data.make_data as md
import entropy.data.selectors as sl
import entropy.data.validators as vl
import entropy.helpers.aws as ha
import entropy.helpers.data as hd
import entropy.helpers.helpers as hh

# new code
sys.path.append("/Users/fgu/dev/projects/src")
import src.config as cf
import src.data.aggregators as ag
import src.data.validators as vl
import src.helpers.data as hd

pd.set_option("display.max_rows", 200)
pd.set_option("display.max_columns", 120)
pd.set_option("max_colwidth", None)
%load_ext autoreload
%autoreload 2

fs = s3fs.S3FileSystem(profile="3di")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
fs.ls("3di-project-entropy")

['3di-project-entropy/analysis_777.csv',
 '3di-project-entropy/analysis_777.parquet',
 '3di-project-entropy/analysis_X77.csv',
 '3di-project-entropy/analysis_X77.parquet',
 '3di-project-entropy/analysis_XX7.csv',
 '3di-project-entropy/analysis_XX7.parquet',
 '3di-project-entropy/analysis_XX7new.parquet',
 '3di-project-entropy/analysis_XX7old.parquet',
 '3di-project-entropy/debug.parquet',
 '3di-project-entropy/entropy_XX0.parquet']

In [ ]:
dft = hd.read_txn_data("111")
hd.inspect(dft)

In [3]:
dfa = hd.read_analysis_data("XX0")
hd.inspect(dfa)

shape: (102,302, 169), users: 3345


,user_id,ym,ymn,month,txns_count,txns_volume,txns_count_spend,txns_count_sa,txns_count_ca,has_benefits,has_od_fees,month_income,year_income,month_income_mean,income_var,has_month_income,inflows,outflows,netflows,netflows_norm,inflows_norm,outflows_norm,has_pos_netflows,pos_netflows,has_inflows,user_reg_ym,month_spend,age,is_female,region,is_urban,region_code,has_savings_account,has_current_account,generation,generation_code,prop_credit,accounts_active,accounts_total,investments,up_savings,ca_transfers,cc_payments,loan_funds,loan_rpmts,dspend,dspend_count,dspend_mean,dspend_clothes,dspend_entertainment,dspend_food,dspend_groceries,dspend_other,dspend_dd,spend_household,spend_insurance,spend_cash,spend_groceries,spend_phone_and_mobile,spend_bank_charges,...,count_entertainment_tv_media,count_energy_and_water,count_taxes,count_public_transport,count_hobbies,count_loan_repayment,count_eating_out,count_personal_care,count_home,count_vehicle_hire,count_holidays,count_clothes_and_shoes,count_appliances,count_entertainment,count_health_care,count_diy,count_vehicle,count_office,count_alcohol,count_sports,count_gifts,count_housing,count_flights,count_business_expenses,count_financial,count_taxi,count_gambling,count_charity,count_children,count_jewellery,count_garden,count_accessories,count_fines,count_education,count_flowers,count_mortgage_release,count_legal,count_pet,count_administration,count_art,count_childcare,entropy_tag,entropy_tag_z,entropy_tag_s,entropy_tag_sz,std_tag,entropy_tag_spend,entropy_tag_spend_z,entropy_tag_spend_s,entropy_tag_spend_sz,std_tag_spend,entropy_merchant,entropy_merchant_z,entropy_merchant_s,entropy_merchant_sz,std_merchant,entropy_groc,entropy_groc_z,entropy_groc_s,entropy_groc_sz
0,1210,2012-02,201202,2,89.0,24168.910156,62,3.0,37.0,0,0,5.20883,59.788048,4.982337,0.458021,1,10.0,2000.0,-1990.0,-382.043579,1.919817,383.963379,0,-0.0,1,2012-01,5.39587,44.0,0.0,north west,1.0,8,True,True,Gen X,2,0.452722,7,15,39.75,0.0,3282.010010,3242.260010,0.0,0.0,1239.489990,25.0,49.579601,66.00,0.00,18.25,382.890015,772.349976,0.0,0.00765,0.94214,0.60,0.38289,0.013,0.11383,...,1.0,1.0,1.0,1.0,0.0,0.0,1.0,4.0,3.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,7.0,0.0,1.0,2.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.680826,1.055741,2.794094,0.695465,6.863753,3.798944,1.230455,4.988456,0.195244,2.71154,3.862898,0.462827,10.026798,-0.006050,0.479624,1.870254,0.309468,3.876838,-0.532226
1,1210,2012-03,201203,3,88.0,20173.070312,61,2.0,39.0,0,0,4.82800,59.788048,4.982337,0.325535,1,10.0,0.0,10.0,2.071251,2.071251,0.000000,1,10.0,1,2012-01,4.19900,44.0,0.0,north west,1.0,8,True,True,Gen X,2,0.275532,6,15,39.75,0.0,3242.350098,2396.600098,0.0,0.0,672.849976,25.0,26.914000,29.99,18.77,12.38,551.710022,60.000000,0.0,0.06000,0.16485,0.76,0.55171,0.013,0.10474,...,2.0,1.0,1.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,8.0,0.0,1.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.525209,0.785321,2.694468,0.327253,7.628528,3.394619,0.735186,4.833017,-0.202743,3.28974,3.612439,0.226937,10.025311,-0.030106,0.519582,1.774397,0.190957,3.779950,-0.911817


In [4]:
dfa.columns

Index(['user_id', 'ym', 'ymn', 'month', 'txns_count', 'txns_volume',
       'txns_count_spend', 'txns_count_sa', 'txns_count_ca', 'has_benefits',
       ...
       'std_tag_spend', 'entropy_merchant', 'entropy_merchant_z',
       'entropy_merchant_s', 'entropy_merchant_sz', 'std_merchant',
       'entropy_groc', 'entropy_groc_z', 'entropy_groc_s', 'entropy_groc_sz'],
      dtype='object', length=169)

In [3]:
dfd = hd.read_debug_data()
hd.inspect(dfd)

shape: (1,043,727, 172), users: 33884


,user_id,ym,ymn,month,txns_count,txns_volume,txns_count_spend,txns_count_sa,txns_count_ca,has_benefits,has_od_fees,month_income,year_income,month_income_mean,income_var,has_month_income,inflows,outflows,netflows,netflows_norm,inflows_norm,outflows_norm,has_pos_netflows,pos_netflows,has_inflows,user_reg_ym,month_spend,age,is_female,region,is_urban,region_code,has_savings_account,has_current_account,generation,generation_code,prop_credit,accounts_active,accounts_total,investments,up_savings,ca_transfers,cc_payments,loan_funds,loan_rpmts,nunique_tag,nunique_tag_spend,nunique_merchant,dspend,dspend_count,dspend_mean,dspend_clothes,dspend_entertainment,dspend_food,dspend_groceries,dspend_other,dspend_dd,spend_household,spend_insurance,spend_cash,...,count_entertainment_tv_media,count_energy_and_water,count_taxes,count_public_transport,count_hobbies,count_loan_repayment,count_eating_out,count_personal_care,count_home,count_vehicle_hire,count_holidays,count_clothes_and_shoes,count_appliances,count_entertainment,count_health_care,count_diy,count_vehicle,count_office,count_alcohol,count_sports,count_gifts,count_housing,count_flights,count_business_expenses,count_financial,count_taxi,count_gambling,count_charity,count_children,count_jewellery,count_garden,count_accessories,count_fines,count_education,count_flowers,count_mortgage_release,count_legal,count_pet,count_administration,count_art,count_childcare,entropy_tag,entropy_tag_z,entropy_tag_s,entropy_tag_sz,std_tag,entropy_tag_spend,entropy_tag_spend_z,entropy_tag_spend_s,entropy_tag_spend_sz,std_tag_spend,entropy_merchant,entropy_merchant_z,entropy_merchant_s,entropy_merchant_sz,std_merchant,entropy_groc,entropy_groc_z,entropy_groc_s,entropy_groc_sz
0,1210,2012-02,201202,2,89.0,24168.910156,62,3.0,37.0,0,0,5.20883,59.788048,4.982337,0.458021,1,10.0,2000.0,-1990.0,-382.043579,1.919817,383.963379,0,-0.0,1,2012-01,5.39587,44.0,0.0,north west,1.0,8,True,True,Gen X,2,0.452722,7,15,39.75,0.0,3282.010010,3242.260010,0.0,0.0,9,22,23,1239.489990,25.0,49.579601,66.00,0.00,18.25,382.890015,772.349976,0.0,0.00765,0.94214,0.60,...,1.0,1.0,1.0,1.0,0.0,0.0,1.0,4.0,3.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,7.0,0.0,1.0,2.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.680826,1.055741,2.794094,0.695465,6.863753,3.798944,1.230455,4.988456,0.195244,2.71154,3.862898,0.462827,10.026798,-0.006050,0.479624,1.870254,0.309468,3.876838,-0.532226
1,1210,2012-03,201203,3,88.0,20173.070312,61,2.0,39.0,0,0,4.82800,59.788048,4.982337,0.325535,1,10.0,0.0,10.0,2.071251,2.071251,0.000000,1,10.0,1,2012-01,4.19900,44.0,0.0,north west,1.0,8,True,True,Gen X,2,0.275532,6,15,39.75,0.0,3242.350098,2396.600098,0.0,0.0,8,19,22,672.849976,25.0,26.914000,29.99,18.77,12.38,551.710022,60.000000,0.0,0.06000,0.16485,0.76,...,2.0,1.0,1.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,8.0,0.0,1.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.525209,0.785321,2.694468,0.327253,7.628528,3.394619,0.735186,4.833017,-0.202743,3.28974,3.612439,0.226937,10.025311,-0.030106,0.519582,1.774397,0.190957,3.779950,-0.911817


In [9]:
vl.min_year_income(dfd)

AssertionError: 

In [21]:
dfd.month_income_mean.min() >= round(cf.MIN_YEAR_INCOME / 12, 2))

False

In [30]:
a = dfd.month_income_mean.min()
b = round(cf.MIN_YEAR_INCOME / 12, 2)

import math

math.isclose(a, b) or (a > b)

False

In [25]:
format(, ".20f")

'0.41999999999999998446'